In [1]:
import pandas as pd
import json
from pandas import json_normalize
from textblob import TextBlob

## Archivo australian_user_reviews.json

In [2]:
# Leer el contenido del archivo australian_user_reviews.json
with open("australian_user_reviews.json", "r", encoding="utf-8") as f:
    data = f.readlines()
    
# Convertir la lineas a registros JSON
records = [eval(line.strip()) for line in data]

In [3]:
# Crear el DataFrame
df = pd.DataFrame(records)

In [4]:
df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Cada registro de la columna reviews esta conformada por una lista que en su interior tiene varios JSON, 
por eso primero se utiliza exploded y luego json_normalize, se trabaja todo sobre una copia de DF.

In [5]:
# Copia de del DataFrame sobre el cual se trabajara
df_copy = df.copy()

In [6]:
# Usar explode para descomponer la lista y replicar filas de la columna 'reviews'
df_exploded = df_copy.explode('reviews', ignore_index=True)

In [7]:
df_exploded

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [8]:
# Normalizacion de JSON a DataFrame  de la columna 'reviews'

In [9]:
df_reviews = json_normalize(df_exploded['reviews'])

In [10]:
df_reviews.tail(5)

,funny,posted,last_edited,item_id,helpful,recommend,review
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D
59332,,Posted July 2.,,440,No ratings yet,True,so much fun :D


In [11]:
# DataFrame sobre el cual se realizara la limpieza
df_user_reviews = df_exploded.join(df_reviews)

df_user_reviews.head(5)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [12]:
# Elimino las columnas 'user_url' y 'reviews' , no sirven 
df_user_reviews = df_user_reviews.drop(columns=['user_url' , 'reviews'])

df_user_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [13]:
# Verificar si hay registros duplicados
hay_duplicados = df_user_reviews.duplicated()

# Mostrar los registros duplicados
df_duplicados = df_user_reviews[hay_duplicados]
df_duplicados

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
1114,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
2894,ImSeriouss,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy"
2895,ImSeriouss,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...
2896,ImSeriouss,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy
2897,ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...
...,...,...,...,...,...,...,...,...
44456,76561198092022514,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
44457,76561198092022514,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44458,76561198092022514,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
44459,76561198092022514,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...


In [14]:
filtro1 = df_user_reviews['item_id'] == '346110'
filtro2 = df_user_reviews['posted'] == 'Posted September 24, 2015.'
registros = df_user_reviews[filtro1 & filtro2]
registros

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
1113,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
1114,bokkkbokkk,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep


In [15]:
# Eliminar los registros totalmente nulos
df_user_reviews = df_user_reviews.dropna(how='all')

# Eliminar los registros duplicados 
df_user_reviews = df_user_reviews.drop_duplicates()

In [16]:
df_user_reviews.tail(5)

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
59328,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D
59332,LydiaMorley,,Posted July 2.,,440,No ratings yet,True,so much fun :D


In [17]:
# Elimino registros con Nan tanto en 'reviews' como en 'item_id'
review_null =  df_user_reviews['review'].isnull()
item_id_null = df_user_reviews['item_id'].isnull()

# Encuentra los índices de las filas que cumplen ambas condiciones
indices_a_eliminar = df_user_reviews[review_null & item_id_null].index

# Elimina las filas con los índices encontrados
df_user_reviews.drop(indices_a_eliminar, inplace=True)

In [18]:
# columna mes y anio, lo separo de este modo porque existen valores sin el anio respectivo
df_user_reviews['mes'] = df_user_reviews['posted'].str.extract(r'(\w+ \d+)')
df_user_reviews['year'] = df_user_reviews['posted'].str.extract(r'(\d{4})')

# Elimino las columnas que no sirven 
df_user_reviews = df_user_reviews.drop(columns=['funny' , 'last_edited' , 'helpful'])
df_user_reviews.drop('posted' , axis = 1, inplace = True)

In [19]:
df_user_reviews

,user_id,item_id,recommend,review,mes,year
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,November 5,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,July 15,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,April 21,2011
3,js41637,251610,True,I know what you think when you see this title ...,June 24,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,September 8,2013
...,...,...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...,July 10,NaN
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...,July 8,NaN
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,July 3,NaN
59331,LydiaMorley,730,True,:D,July 20,NaN


### aqui empieza el analisis de sensibilidad

In [20]:
# Reemplaza los valores vacíos de la columna 'review' por 'neutral' 
df_user_reviews['review'].fillna('neutral', inplace=True)

Debido a una parte del enuncuado que dice: 
"De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1"
reemplazando los nulos por la palabra neutral, durante el analisis de sensibilidad se le dara un valor de 1 

In [21]:
# Cambia la columna 'review' a tipo str
df_user_reviews['review'] = df_user_reviews['review'].astype(str)

El paso de arriba nose si es necesario, tuve que hacerlo porque estaba usando nltk pero marcaba error.
use la libreria TextBlob y se soluciono, despues de intentar solucionar los problemas de nltk

In [23]:
# Libreria para el analisis

# Definir una función para analizar el sentimiento
def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    # Obtener la polaridad del sentimiento (-1 a 1, donde -1 es negativo, 0 es neutral y 1 es positivo)
    polaridad = analysis.sentiment.polarity
    if polaridad > 0:
        return 2
    elif polaridad < 0:
        return 0
    else:
        return 1

In [25]:
# se crea una columna 'sentiment_analysis'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analizar_sentimiento)

In [26]:
# Elimino la columna 'review' y 'mes'
df_user_reviews.drop('review' , axis = 1, inplace = True)
df_user_reviews.drop('mes' , axis = 1, inplace = True)

### DataFrame 

In [27]:
df_user_reviews.head(5)

,user_id,item_id,recommend,year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0


In [28]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58431 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   item_id             58431 non-null  object
 2   recommend           58431 non-null  object
 3   year                48498 non-null  object
 4   sentiment_analysis  58431 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


sentiment_analysis:
positivo = 2 , neutral = 1 , negativo = 0

In [29]:
# Utiliza to_csv() para guardar el DataFrame como un archivo CSV
# df_user_reviews.to_csv('user_reviews.csv', index=False)

## Archivo output_steam_games.json

In [30]:
# Leer el contenido del archivo output_steam_games.json

data = []
with open('output_steam_games.json') as f:
    for line in f:
        data.append(json.loads(line))

In [31]:
df_steam = pd.DataFrame(data)

In [32]:
df_steam.tail(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,False,773640,NaN,"Nikita ""Ghost_RUS""",NaN,NaN,NaN,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,NaN,Sacada,NaN,NaN,NaN,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,NaN,NaN,NaN,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns",NaN,NaN,NaN,NaN
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  object 
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  object 
 17  items           88310 non-nul

In [34]:
# Usar explode para descomponer la lista y replicar filas de la columna 'tags'
df_steam_exploded = df_steam.explode('tags', ignore_index=True)

In [35]:
# Usar explode para descomponer la lista y replicar filas de la columna 'specs'
df_steam_exploded = df_steam_exploded.explode('specs', ignore_index=True)

In [36]:
# Usar explode para descomponer la lista y replicar filas de la columna 'items'
#df_steam_exploded = df_steam_exploded.explode('items', ignore_index=True)

In [37]:
# Elimino los registros sin Genero
df_steam_exploded.dropna(subset=['tags'], inplace=True)

In [38]:
# Elimino la columna 'genres'
df_steam_exploded.drop('genres', axis = 1, inplace = True)

In [39]:
# Reemplazo 'tags' por 'genres'
df_steam_exploded.rename(columns={'tags': 'genres'}, inplace=True)

In [40]:
df_steam_exploded.head(5)

,publisher,app_name,title,url,release_date,genres,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
88310,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Strategy,http://steamcommunity.com/app/761140/reviews/?...,4.49,Single-player,4.99,False,761140,NaN,Kotoshiro,NaN,NaN,NaN,NaN
88311,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Action,http://steamcommunity.com/app/761140/reviews/?...,4.49,Single-player,4.99,False,761140,NaN,Kotoshiro,NaN,NaN,NaN,NaN
88312,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Indie,http://steamcommunity.com/app/761140/reviews/?...,4.49,Single-player,4.99,False,761140,NaN,Kotoshiro,NaN,NaN,NaN,NaN
88313,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Casual,http://steamcommunity.com/app/761140/reviews/?...,4.49,Single-player,4.99,False,761140,NaN,Kotoshiro,NaN,NaN,NaN,NaN
88314,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,Simulation,http://steamcommunity.com/app/761140/reviews/?...,4.49,Single-player,4.99,False,761140,NaN,Kotoshiro,NaN,NaN,NaN,NaN


In [41]:
df_steam_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786858 entries, 88310 to 875420
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   publisher       615129 non-null  object 
 1   app_name        786854 non-null  object 
 2   title           703826 non-null  object 
 3   url             786858 non-null  object 
 4   release_date    703378 non-null  object 
 5   genres          786858 non-null  object 
 6   reviews_url     786758 non-null  object 
 7   discount_price  7087 non-null    float64
 8   specs           784951 non-null  object 
 9   price           762599 non-null  object 
 10  early_access    786858 non-null  object 
 11  id              786758 non-null  object 
 12  metascore       152034 non-null  object 
 13  developer       697846 non-null  object 
 14  user_id         0 non-null       object 
 15  steam_id        0 non-null       object 
 16  items           0 non-null       object 
 17  items_

In [42]:
# Seleccion de columnas que voy a necesitar 
columns = ['genres' , 'title' , 'release_date' , 'price' , 'specs' ,  'id' ]
df_steam_exploded = df_steam_exploded[columns]

In [43]:
df_steam_exploded

,genres,title,release_date,price,specs,id
88310,Strategy,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88311,Action,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88312,Indie,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88313,Casual,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88314,Simulation,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
...,...,...,...,...,...,...
875416,VR,NaN,NaN,4.99,HTC Vive,681550
875417,VR,NaN,NaN,4.99,Oculus Rift,681550
875418,VR,NaN,NaN,4.99,Tracked Motion Controllers,681550
875419,VR,NaN,NaN,4.99,Standing,681550


In [44]:
# Eliminar los registros totalmente nulos
df_steam_exploded = df_steam_exploded.dropna(how='all')

# Eliminar los registros duplicados 
df_steam_exploded = df_steam_exploded.drop_duplicates()

In [45]:
df_steam_exploded

,genres,title,release_date,price,specs,id
88310,Strategy,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88311,Action,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88312,Indie,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88313,Casual,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
88314,Simulation,Lost Summoner Kitty,2018-01-04,4.99,Single-player,761140
...,...,...,...,...,...,...
875416,VR,NaN,NaN,4.99,HTC Vive,681550
875417,VR,NaN,NaN,4.99,Oculus Rift,681550
875418,VR,NaN,NaN,4.99,Tracked Motion Controllers,681550
875419,VR,NaN,NaN,4.99,Standing,681550


In [46]:
# Eliminno los registros sin anios
df_steam_games = df_steam_exploded.copy()
df_steam_games.dropna(subset=['release_date'], inplace=True)


In [47]:
# Agrego la columna 'year' y elimino 'release_date'
df_steam_games['year'] = df_steam_games['release_date'].str.extract(r'(\d{4})')
df_steam_games.drop('release_date', axis = 1, inplace = True)

In [48]:
# Cambio de nombre la columna 'id' por 'item_id'
df_steam_games.rename(columns={'id': 'item_id'}, inplace=True) 

### DataFrame para el EDA

In [49]:
df_steam_games

,genres,title,price,specs,item_id,year
88310,Strategy,Lost Summoner Kitty,4.99,Single-player,761140,2018
88311,Action,Lost Summoner Kitty,4.99,Single-player,761140,2018
88312,Indie,Lost Summoner Kitty,4.99,Single-player,761140,2018
88313,Casual,Lost Summoner Kitty,4.99,Single-player,761140,2018
88314,Simulation,Lost Summoner Kitty,4.99,Single-player,761140,2018
...,...,...,...,...,...,...
875368,Atmospheric,EXIT 2 - Directions,4.99,Steam Achievements,658870,2017
875369,Atmospheric,EXIT 2 - Directions,4.99,Steam Cloud,658870,2017
875370,Relaxing,EXIT 2 - Directions,4.99,Single-player,658870,2017
875371,Relaxing,EXIT 2 - Directions,4.99,Steam Achievements,658870,2017


In [50]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 703298 entries, 88310 to 875372
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   genres   703298 non-null  object
 1   title    703294 non-null  object
 2   price    682287 non-null  object
 3   specs    701391 non-null  object
 4   item_id  703198 non-null  object
 5   year     700791 non-null  object
dtypes: object(6)
memory usage: 37.6+ MB


In [52]:
df_steam_games['price'] = df_steam_games['price'].astype(str)

In [53]:
# Utiliza to_parquet() para guardar el DataFrame como un archivo CSV
df_steam_games.to_parquet('steam_games.parquet', index=False)

## Archivo australian_users_items.json

In [27]:
# Leer el contenido del archivo australian_users_items.json
with open('australian_users_items.json', 'r', encoding='utf-8') as f:
    data = f.readlines()

records = [eval(line.strip()) for line in data]

In [28]:
df_user_items = pd.DataFrame(records)

In [29]:
df_user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [30]:
# Usar explode para descomponer la lista y replicar filas de la columna 'items'
df_user_items =  df_user_items.explode('items', ignore_index=True)

In [32]:
# Normalizacion de JSON a DataFrame  de la columna 'items'
df_items2 = json_normalize(df_user_items['items'])

In [33]:
# DataFrame sobre el cual se realizara el trabajo
df_user_items = df_user_items.join(df_items2)

In [34]:
df_user_items = df_user_items.drop(columns = 'items', axis =1)

In [35]:
# Eliminar los registros totalmente nulos
df_user_items = df_user_items.dropna(how='all')

# Eliminar los registros duplicados 
df_user_items = df_user_items.drop_duplicates()

In [36]:
# Elimino las columnas que no voy a usar
df_user_items = df_user_items.drop(columns = ['item_name' , 'items_count' , 'user_url'])

In [37]:
df_user_items = df_user_items.drop(columns = ['steam_id' , 'playtime_2weeks'] , axis =1)

In [38]:
# Elimino registros con Nan tanto en 'playtime_forever' como en 'item_id'
time_null =  df_user_items['playtime_forever'].isnull()
item_id_null = df_user_items['item_id'].isnull()

# Encuentra los índices de las filas que cumplen ambas condiciones
indices_a_eliminar = df_user_items[time_null & item_id_null].index

# Elimina las filas con los índices encontrados
df_user_items.drop(indices_a_eliminar, inplace=True)

In [39]:
df_user_items = df_user_items.query('playtime_forever != 0.0')

In [40]:
df_user_items = df_user_items.dropna()

In [41]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3246375 entries, 0 to 5170013
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   playtime_forever  float64
dtypes: float64(1), object(2)
memory usage: 99.1+ MB


In [42]:
df_user_items

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
2,76561197970982479,30,7.0
8,76561197970982479,300,4733.0
9,76561197970982479,240,1853.0
10,76561197970982479,3830,333.0
...,...,...,...
5170007,76561198329548331,304930,677.0
5170008,76561198329548331,227940,43.0
5170011,76561198329548331,388490,3.0
5170012,76561198329548331,521570,4.0


In [127]:
# Utiliza to_csv() para guardar el DataFrame como un archivo CSV
# df_user_items.to_csv('user_items.csv', index=False)

### Combinacion de df_steam_games y df_user_items

In [79]:
df_user_time = df_user_items.copy()

In [80]:
# Elimino las columnas innecesarias
df_user_time.drop(columns = 'item_name' , axis = 1 , inplace = True)

In [81]:
df_user_time.drop('user_id', axis = 1, inplace = True)

In [82]:
# Elimino las registros innecesarias
df_user_time.dropna(subset=['playtime_forever'], inplace=True)

In [83]:
df_user_time.drop(df_user_time[df_user_time['playtime_forever'] == 0].index, inplace=True)


In [84]:
df_user_time

,item_id,playtime_forever
0,10,6.0
2,30,7.0
8,300,4733.0
9,240,1853.0
10,3830,333.0
...,...,...
5170007,304930,677.0
5170008,227940,43.0
5170011,388490,3.0
5170012,521570,4.0


In [104]:
df_agrupado = df_user_time.groupby('item_id')['playtime_forever'].sum().reset_index()
df_agrupado

,item_id,playtime_forever
0,10,17107858.0
1,20,960524.0
2,30,756375.0
3,40,154424.0
4,50,726545.0
...,...,...
10045,527570,2.0
10046,527810,2.0
10047,527890,1.0
10048,527900,44.0


In [112]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_id           10050 non-null  int64  
 1   playtime_forever  10050 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 157.2 KB


In [117]:
# Vuelvo a convertirlo en str para concatenar 
df_agrupado['item_id'] = df_agrupado['item_id'].astype(str)

In [118]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_id           10050 non-null  object 
 1   playtime_forever  10050 non-null  float64
dtypes: float64(1), object(1)
memory usage: 157.2+ KB


In [106]:
# tabajo sobre una copia de df_steam_games para las modificaciones
df_steam_games_mod = df_steam_games.copy()

In [107]:
# Elimino las columnas innecesarias
df_steam_games_mod.drop(columns = 'specs' , axis = 1 , inplace = True)

In [108]:
df_steam_games_mod.drop(columns = 'price' , axis = 1 , inplace = True)

In [109]:
# Eliminar los registros duplicados 
df_steam_games_mod = df_steam_games_mod.drop_duplicates()

In [119]:
# Utiliza la función merge() para combinar los DataFrames en base a la columna 'items_id'
# El parámetro 'on' especifica la columna en la que se basará la combinación.
# El parámetro 'how' especifica el tipo de combinación, en este caso, 'inner' significa que se conservarán solo las filas comunes.
# Puedes ajustar 'how' según tus necesidades (inner, outer, left, right).

merged_df = df_steam_games_mod.merge(df_agrupado, on='item_id', how='inner')

# merged_df ahora contendrá las filas comunes entre df_steam_games y df_user_items


In [120]:
merged_df

,genres,title,item_id,year,playtime_forever
0,Racing,Carmageddon Max Pack,282010,1997,9319.0
1,Action,Carmageddon Max Pack,282010,1997,9319.0
2,Classic,Carmageddon Max Pack,282010,1997,9319.0
3,Indie,Carmageddon Max Pack,282010,1997,9319.0
4,Gore,Carmageddon Max Pack,282010,1997,9319.0
...,...,...,...,...,...
72762,Survival,Counter-Strike: Condition Zero,80,2004,2469131.0
72763,Atmospheric,Counter-Strike: Condition Zero,80,2004,2469131.0
72764,Dark,Counter-Strike: Condition Zero,80,2004,2469131.0
72765,Simulation,Counter-Strike: Condition Zero,80,2004,2469131.0


In [46]:
# Eliminar los registros duplicados 
# merged_df = merged_df.drop_duplicates()

In [121]:
# Elimino las filas con valores NaN 
merged_df = merged_df.dropna()


### DF para la etapa de funciones de API

In [122]:
merged_df

,genres,title,item_id,year,playtime_forever
0,Racing,Carmageddon Max Pack,282010,1997,9319.0
1,Action,Carmageddon Max Pack,282010,1997,9319.0
2,Classic,Carmageddon Max Pack,282010,1997,9319.0
3,Indie,Carmageddon Max Pack,282010,1997,9319.0
4,Gore,Carmageddon Max Pack,282010,1997,9319.0
...,...,...,...,...,...
72762,Survival,Counter-Strike: Condition Zero,80,2004,2469131.0
72763,Atmospheric,Counter-Strike: Condition Zero,80,2004,2469131.0
72764,Dark,Counter-Strike: Condition Zero,80,2004,2469131.0
72765,Simulation,Counter-Strike: Condition Zero,80,2004,2469131.0


In [123]:
# Utiliza to_csv() para guardar el DataFrame como un archivo CSV
merged_df.to_csv('merged_df.csv', index=False)

### Combinacion de df_user_reviews y df_user_items

In [43]:
df_user_items

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
2,76561197970982479,30,7.0
8,76561197970982479,300,4733.0
9,76561197970982479,240,1853.0
10,76561197970982479,3830,333.0
...,...,...,...
5170007,76561198329548331,304930,677.0
5170008,76561198329548331,227940,43.0
5170011,76561198329548331,388490,3.0
5170012,76561198329548331,521570,4.0


In [44]:
df_user_reviews

,user_id,item_id,recommend,year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0
...,...,...,...,...,...
59328,76561198312638244,70,True,NaN,2
59329,76561198312638244,362890,True,NaN,2
59330,LydiaMorley,273110,True,NaN,2
59331,LydiaMorley,730,True,NaN,2


In [45]:
# Utiliza la función merge() para combinar los DataFrames en base a la columna 'items_id'

merged_df_2 = df_user_reviews.merge(df_user_items, on=['user_id' , 'item_id'], how='inner')

### DF para la etapa de funciones de API

In [47]:
merged_df_2.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44100 entries, 0 to 44099
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             44100 non-null  object 
 1   item_id             44100 non-null  object 
 2   recommend           44100 non-null  object 
 3   year                36210 non-null  object 
 4   sentiment_analysis  44100 non-null  int64  
 5   playtime_forever    44100 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 2.4+ MB


In [48]:
# Utiliza to_csv() para guardar el DataFrame como un archivo CSV
merged_df_2.to_csv('merged_df_2.csv', index=False)